In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/application_test.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/application_train.csv


In [3]:
train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')

In [4]:
def agg_numeric(df, group_var, df_name):

    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    
    # Remove the columns with all redundant values
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    return agg

In [5]:
def count_categorical(df, group_var, df_name):
 
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

## Bureau

In [6]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

bureau_CREDIT_TYPE_Mobile operator loan_count_norm  \
SK_ID_CURR                                                       
100001                                                    0.0    
100002                                                    0.0    
100003                                                    0.0    
100004                                                    0.0    
100005                                                    0.0    

            bureau_CREDIT_TYPE_Mobile operator loan_count  \
SK_ID_CURR                                                  
100001                                                  0   
100002                                                  0   
100003                                                  0   
100004                                                  0   
100005                                                  0   

            bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count_norm  \
SK_ID_CURR                                                                               
100001                                                    0.0                            
100002                                                    0.0                            
100003                                                    0.0                            
100004                                                    0.0                            
100005                                                    0.0                            

            bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count  \
SK_ID_CURR                                                                          
100001                                                      0                       
100002                                                      0                       
100003                                                      0                       
100004                                                      0                       
100005                                                      0                       

            bureau_CREDIT_ACTIVE_Bad debt_count_norm  \
SK_ID_CURR                                             
100001                                           0.0   
100002                                           0.0   
100003                                           0.0   
100004                                           0.0   
100005                                           0.0   

            bureau_CREDIT_ACTIVE_Bad debt_count  \
SK_ID_CURR                                        
100001                                        0   
100002                                        0   
100003                                        0   
100004                                        0   
100005                                        0   

            bureau_CREDIT_TYPE_Interbank credit_count_norm  \
SK_ID_CURR                                                   
100001                                                 0.0   
100002                                                 0.0   
100003                                                 0.0   
100004                                                 0.0   
100005                                                 0.0   

            bureau_CREDIT_TYPE_Interbank credit_count  \
SK_ID_CURR                                              
100001                                              0   
100002                                              0   
100003                                              0   
100004                                              0   
100005                                              0   

            bureau_CREDIT_TYPE_Real estate loan_count_norm  \
SK_ID_CURR                                                   
100001                                                 0.0   
100002                                                 0.0   
100003                                                 0.0  

In [7]:
bureau_agg = agg_numeric(bureau.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


bureau_DAYS_CREDIT_sum  bureau_DAYS_ENDDATE_FACT_sum  \
0                   -5145                       -3302.0   
1                   -6992                       -4185.0   
2                   -5603                       -3292.0   
3                   -1734                       -1065.0   
4                    -572                        -123.0   

   bureau_DAYS_CREDIT_min  bureau_DAYS_CREDIT_ENDDATE_min  \
0                   -1572                         -1329.0   
1                   -1437                         -1072.0   
2                   -2586                         -2434.0   
3                   -1326                          -595.0   
4                    -373                          -128.0   

   bureau_DAYS_ENDDATE_FACT_min  bureau_DAYS_ENDDATE_FACT_mean  \
0                       -1328.0                    -825.500000   
1                       -1185.0                    -697.500000   
2                       -2131.0                   -1097.333333   
3                        -683.0                    -532.500000   
4                        -123.0                    -123.000000   

   bureau_DAYS_CREDIT_mean  bureau_DAYS_CREDIT_UPDATE_sum  \
0              -735.000000                           -652   
1              -874.000000                          -3999   
2             -1400.750000                          -3264   
3              -867.000000                          -1064   
4              -190.666667                           -163   

   bureau_DAYS_ENDDATE_FACT_max  bureau_DAYS_CREDIT_UPDATE_min  ...  \
0                        -544.0                           -155  ...   
1                         -36.0                          -1185  ...   
2                        -540.0                          -2131  ...   
3                        -382.0                           -682  ...   
4                        -123.0                           -121  ...   

   bureau_AMT_CREDIT_SUM_min  SK_ID_CURR  bureau_AMT_CREDIT_SUM_mean  \
0                    85500.0      100001               207623.571429   
1                        0.0      100002               108131.945625   
2                    22248.0      100003               254350.125000   
3                    94500.0      100004                94518.900000   
4                    29826.0      100005               219042.000000   

   bureau_AMT_CREDIT_SUM_DEBT_max  bureau_AMT_CREDIT_SUM_max  \
0                        373239.0                   378000.0   
1                        245781.0                   450000.0   
2                             0.0                   810000.0   
3                             0.0                    94537.8   
4                        543087.0                   568800.0   

   bureau_AMT_CREDIT_SUM_DEBT_sum  bureau_AMT_CREDIT_SUM_sum  \
0                        596686.5                1453365.000   
1                        245781.0                 865055.565   
2                             0.0                1017400.500   
3                             0.0                 189037.800   
4                        568408.5                 657126.000   

   bureau_AMT_CREDIT_MAX_OVERDUE_min  bureau_AMT_CREDIT_MAX_OVERDUE_mean  \
0                                NaN                                 NaN   
1                                0.0                            1681.029   
2                                0.0                               0.000   
3                                0.0                               0.000   
4                                0.0                               0.000   

   bureau_AMT_CREDIT_MAX_OVERDUE_max  
0                                NaN  
1                           5043.645  
2                              0.000  
3                              0.000  
4                              0.000  

[5 rows x 57 columns]

##  Bureau Balance 

In [8]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

bureau_balance_STATUS_4_count_norm  \
SK_ID_BUREAU                                       
5001709                                      0.0   
5001710                                      0.0   
5001711                                      0.0   
5001712                                      0.0   
5001713                                      0.0   

              bureau_balance_STATUS_3_count_norm  \
SK_ID_BUREAU                                       
5001709                                      0.0   
5001710                                      0.0   
5001711                                      0.0   
5001712                                      0.0   
5001713                                      0.0   

              bureau_balance_STATUS_2_count_norm  \
SK_ID_BUREAU                                       
5001709                                      0.0   
5001710                                      0.0   
5001711                                      0.0   
5001712                                      0.0   
5001713                                      0.0   

              bureau_balance_STATUS_5_count_norm  \
SK_ID_BUREAU                                       
5001709                                      0.0   
5001710                                      0.0   
5001711                                      0.0   
5001712                                      0.0   
5001713                                      0.0   

              bureau_balance_STATUS_4_count  bureau_balance_STATUS_3_count  \
SK_ID_BUREAU                                                                 
5001709                                   0                              0   
5001710                                   0                              0   
5001711                                   0                              0   
5001712                                   0                              0   
5001713                                   0                              0   

              bureau_balance_STATUS_2_count  bureau_balance_STATUS_5_count  \
SK_ID_BUREAU                                                                 
5001709                                   0                              0   
5001710                                   0                              0   
5001711                                   0                              0   
5001712                                   0                              0   
5001713                                   0                              0   

              bureau_balance_STATUS_1_count_norm  \
SK_ID_BUREAU                                       
5001709                                      0.0   
5001710                                      0.0   
5001711                                      0.0   
5001712                                      0.0   
5001713                                      0.0   

              bureau_balance_STATUS_1_count  \
SK_ID_BUREAU                                  
5001709                                   0   
5001710                                   0   
5001711                                   0   
5001712                                   0   
5001713                                   0   

              bureau_balance_STATUS_0_count_norm  \
SK_ID_BUREAU                                       
5001709                                 0.000000   
5001710                                 0.060241   
5001711                                 0.750000   
5001712                                 0.526316   
5001713                                 0.000000   

              bureau_balance_STATUS_0_count  \
SK_ID_BUREAU                                  
5001709                                   0   
5001710                                   5   
5001711                                   3   
5001712                                  10   
5001713                                   0   

              bureau_balance_STATUS_X_count_norm  \
SK_ID_BUREAU      

In [9]:
bureau_balance_agg = agg_numeric(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_agg.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


bureau_balance_MONTHS_BALANCE_sum  bureau_balance_MONTHS_BALANCE_min  \
0                              -4656                                -96   
1                              -3403                                -82   
2                                 -6                                 -3   
3                               -171                                -18   
4                               -231                                -21   

   bureau_balance_MONTHS_BALANCE_mean  bureau_balance_MONTHS_BALANCE_max  \
0                               -48.0                                  0   
1                               -41.0                                  0   
2                                -1.5                                  0   
3                                -9.0                                  0   
4                               -10.5                                  0   

   bureau_balance_MONTHS_BALANCE_count  SK_ID_BUREAU  
0                                   97       5001709  
1                                   83       5001710  
2                                    4       5001711  
3                                   19       5001712  
4                                   22       5001713

In [10]:
# Dataframe grouped by the loan
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge to include the SK_ID_CURR
bureau_by_loan = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_by_loan, on = 'SK_ID_BUREAU', how = 'left')

# Aggregate the stats for each client
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')

## Merge

In [11]:
# Merge with the value counts of bureau
train = train.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
train = train.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the monthly information grouped by client
train = train.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [12]:
bureau_counts

bureau_CREDIT_TYPE_Mobile operator loan_count_norm  \
SK_ID_CURR                                                       
100001                                                    0.0    
100002                                                    0.0    
100003                                                    0.0    
100004                                                    0.0    
100005                                                    0.0    
...                                                       ...    
456249                                                    0.0    
456250                                                    0.0    
456253                                                    0.0    
456254                                                    0.0    
456255                                                    0.0    

            bureau_CREDIT_TYPE_Mobile operator loan_count  \
SK_ID_CURR                                                  
100001                                                  0   
100002                                                  0   
100003                                                  0   
100004                                                  0   
100005                                                  0   
...                                                   ...   
456249                                                  0   
456250                                                  0   
456253                                                  0   
456254                                                  0   
456255                                                  0   

            bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count_norm  \
SK_ID_CURR                                                                               
100001                                                    0.0                            
100002                                                    0.0                            
100003                                                    0.0                            
100004                                                    0.0                            
100005                                                    0.0                            
...                                                       ...                            
456249                                                    0.0                            
456250                                                    0.0                            
456253                                                    0.0                            
456254                                                    0.0                            
456255                                                    0.0                            

            bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count  \
SK_ID_CURR                                                                          
100001                                                      0                       
100002                                                      0                       
100003                                                      0                       
100004                                                      0                       
100005                                                      0                       
...                                                       ...                       
456249                                                      0                       
456250                                                      0                       
456253                                                      0                       
456254                                                      0                       
456255                                                      0                       

            bureau_CREDIT_ACTIVE_Bad debt_count_norm  \
SK_ID_CURR                

In [15]:
train.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   ...  client_bureau_balance_STATUS_X_count_max  \
0  ...                                       3.0   
1  ...                                       NaN   
2  ...                                       NaN   
3  ...                                       NaN   
4  ...                                       NaN   

  client_bureau_balance_STATUS_0_count_max  \
0                                     18.0   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

  client_bureau_balance_STATUS_C_count_mean  \
0                                     2.875   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

  client_bureau_balance_MONTHS_BALANCE_count_mean  \
0                                           13.75   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

  client_bureau_balance_STATUS_X_count_sum  \
0                                     15.0   
1                                      0.0   
2                                      0.0   
3                                      NaN   
4                                      0.0   

  client_bureau_balance_STATUS_0_count_sum  \
0                                     45.0   
1                                      0.0   
2                                      0.0   
3                                      NaN   
4                                      0.0   

   client_bureau_balance_STATUS_C_count_max  \
0                                      13.0   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

   client_bureau_balance_MONTHS_BALANCE_count_max  \
0                                            22.0   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

   client_bureau_balance_STATUS_C_count_sum  \
0                                      23.0   
1                                       0.0   
2                                       0.0   
3                                       NaN   
4                                       0.0   

   client_bureau_balance_MONTHS_BALANCE_count_sum  
0                                           110.0  
1                                             0.0  
2                                             0.0  
3                                             NaN  
4                                             0.0  

[5 rows x 309 columns]

In [16]:
df = pd.get_dummies(train)
list(df.columns) 

['SK_ID_CURR',
 'TARGET',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAR

# Model training

In [24]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import gc

import matplotlib.pyplot as plt

In [25]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
    

    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [26]:
# Read in the test dataframe
test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

# Merge with the value counts of bureau
test = test.merge(bureau_counts, on = 'SK_ID_CURR', how = 'left')

# Merge with the stats of bureau
test = test.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')

# Merge with the value counts of bureau balance
test = test.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')

In [27]:
train_labels = train['TARGET']

# Align the dataframes, this will remove the 'TARGET' column
train, test = train.align(test, join = 'inner', axis = 1)

train['TARGET'] = train_labels

In [28]:
submission_raw, fi_raw, metrics_raw = model(train,test)

Training Data Shape:  (307511, 428)
Testing Data Shape:  (48744, 428)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.810385	train's binary_logloss: 0.537173	valid's auc: 0.766471	valid's binary_logloss: 0.552794
Early stopping, best iteration is:
[293]	train's auc: 0.827287	train's binary_logloss: 0.519987	valid's auc: 0.767203	valid's binary_logloss: 0.541991
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.810288	train's binary_logloss: 0.537072	valid's auc: 0.767523	valid's binary_logloss: 0.557044
Early stopping, best iteration is:
[265]	train's auc: 0.822264	train's binary_logloss: 0.525006	valid's auc: 0.768109	valid's binary_logloss: 0.549332
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.811653	train's binary_logloss: 0.535599	valid's auc: 0.761028	valid's binary_logloss: 0.553526
Early stopping, best iteration is:
[267]	train's auc: 0.824147	train's binary_logloss: 0.522782	valid's auc: 0.761626	valid's binary_logloss: 0.545557
Training unti

In [29]:
submission_raw.to_csv('test_one.csv', index = False)
#0.75686

In [23]:
train.head()

SK_ID_CURR NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0      100002         Cash loans           M            N               Y   
1      100003         Cash loans           F            N               N   
2      100004    Revolving loans           M            Y               Y   
3      100006         Cash loans           F            N               Y   
4      100007         Cash loans           M            N               Y   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          202500.0    406597.5      24700.5         351000.0   
1             0          270000.0   1293502.5      35698.5        1129500.0   
2             0           67500.0    135000.0       6750.0         135000.0   
3             0          135000.0    312682.5      29686.5         297000.0   
4             0          121500.0    513000.0      21865.5         513000.0   

   ... client_bureau_balance_STATUS_0_count_max  \
0  ...                                     18.0   
1  ...                                      NaN   
2  ...                                      NaN   
3  ...                                      NaN   
4  ...                                      NaN   

  client_bureau_balance_STATUS_C_count_mean  \
0                                     2.875   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

  client_bureau_balance_MONTHS_BALANCE_count_mean  \
0                                           13.75   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

  client_bureau_balance_STATUS_X_count_sum  \
0                                     15.0   
1                                      0.0   
2                                      0.0   
3                                      NaN   
4                                      0.0   

  client_bureau_balance_STATUS_0_count_sum  \
0                                     45.0   
1                                      0.0   
2                                      0.0   
3                                      NaN   
4                                      0.0   

   client_bureau_balance_STATUS_C_count_max  \
0                                      13.0   
1                                       NaN   
2                                       NaN   
3                                       NaN   
4                                       NaN   

   client_bureau_balance_MONTHS_BALANCE_count_max  \
0                                            22.0   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

   client_bureau_balance_STATUS_C_count_sum  \
0                                      23.0   
1                                       0.0   
2                                       0.0   
3                                       NaN   
4                                       0.0   

   client_bureau_balance_MONTHS_BALANCE_count_sum  TARGET  
0                                           110.0       1  
1                                             0.0       0  
2                                             0.0       0  
3                                             NaN       0  
4                                             0.0       0  

[5 rows x 309 columns]

In [24]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df, print_info = False):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        if print_info:
            # Print some summary information
            print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
                "There are " + str(mis_val_table_ren_columns.shape[0]) +
                  " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [25]:
def remove_missing_columns(train, test, threshold = 90):
    # Calculate missing stats for train and test (remember to calculate a percent!)
    train_miss = pd.DataFrame(train.isnull().sum())
    train_miss['percent'] = 100 * train_miss[0] / len(train)
    
    test_miss = pd.DataFrame(test.isnull().sum())
    test_miss['percent'] = 100 * test_miss[0] / len(test)
    
    # list of missing columns for train and test
    missing_train_columns = list(train_miss.index[train_miss['percent'] > threshold])
    missing_test_columns = list(test_miss.index[test_miss['percent'] > threshold])
    
    # Combine the two lists together
    missing_columns = list(set(missing_train_columns + missing_test_columns))
    
    # Print information
    print('There are %d columns with greater than %d%% missing values.' % (len(missing_columns), threshold))
    
    # Drop the missing columns and return
    train = train.drop(columns = missing_columns)
    test = test.drop(columns = missing_columns)
    
    return train, test

In [26]:
bureau_processed = bureau_counts.merge(bureau_agg, on = 'SK_ID_CURR', how = 'left')
bureau_processed_2 = bureau_processed.merge(bureau_balance_by_client, on = 'SK_ID_CURR', how = 'left')
#bureau_processed_2.to_csv('train_bureau.csv', index = False)

In [27]:
bureau_processed_3, test = remove_missing_columns(bureau_processed_2, test,80)

There are 0 columns with greater than 80% missing values.


In [31]:
bureau_processed_3.to_csv('train_bureau2.csv', index = False)
test.to_csv('test_bureau.csv',index=False)